# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 17 2022 10:16AM,248722,15,8657493,"Brookfield Pharmaceuticals, LLC",Released
1,Oct 17 2022 10:16AM,248722,15,8658006,"Brookfield Pharmaceuticals, LLC",Released
2,Oct 17 2022 10:16AM,248722,15,8658007,"Brookfield Pharmaceuticals, LLC",Released
3,Oct 17 2022 10:16AM,248722,15,8658008,"Brookfield Pharmaceuticals, LLC",Released
4,Oct 17 2022 10:16AM,248722,15,8658009,"Brookfield Pharmaceuticals, LLC",Released
5,Oct 17 2022 10:16AM,248722,15,8658010,"Brookfield Pharmaceuticals, LLC",Released
6,Oct 17 2022 10:16AM,248722,15,8658015,"Brookfield Pharmaceuticals, LLC",Released
7,Oct 17 2022 10:16AM,248722,15,8658016,"Brookfield Pharmaceuticals, LLC",Released
8,Oct 17 2022 10:16AM,248722,15,8658017,"Brookfield Pharmaceuticals, LLC",Released
9,Oct 17 2022 10:16AM,248722,15,8658018,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
14,248723,Released,59
15,248724,Released,1
16,248725,Released,1
17,248728,Released,1
18,248729,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248723,NaN,59.0
248724,NaN,1.0
248725,NaN,1.0
248728,NaN,1.0
248729,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248697,0.0,1.0
248698,0.0,6.0
248699,0.0,7.0
248702,0.0,6.0
248704,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248697,0,1
248698,0,6
248699,0,7
248702,0,6
248704,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248697,0,1
1,248698,0,6
2,248699,0,7
3,248702,0,6
4,248704,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248697,,1
1,248698,,6
2,248699,,7
3,248702,,6
4,248704,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC"
21,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation
80,Oct 17 2022 10:12AM,248729,19,"GCH Granules USA, Inc."
81,Oct 17 2022 10:11AM,248719,15,"Alliance Pharma, Inc."
95,Oct 17 2022 10:10AM,248728,19,"NAPP Technologies, LLC"
96,Oct 17 2022 10:06AM,248725,20,"Exact-Rx, Inc."
97,Oct 17 2022 10:02AM,248724,12,Hush Hush
98,Oct 17 2022 9:51AM,248712,10,ISDIN Corporation
151,Oct 17 2022 9:40AM,248718,20,"Emersa Waterbox, LLC"
152,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC",,21
1,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation,,59
2,Oct 17 2022 10:12AM,248729,19,"GCH Granules USA, Inc.",,1
3,Oct 17 2022 10:11AM,248719,15,"Alliance Pharma, Inc.",,14
4,Oct 17 2022 10:10AM,248728,19,"NAPP Technologies, LLC",,1
5,Oct 17 2022 10:06AM,248725,20,"Exact-Rx, Inc.",,1
6,Oct 17 2022 10:02AM,248724,12,Hush Hush,,1
7,Oct 17 2022 9:51AM,248712,10,ISDIN Corporation,,53
8,Oct 17 2022 9:40AM,248718,20,"Emersa Waterbox, LLC",,1
9,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC",21,
1,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation,59,
2,Oct 17 2022 10:12AM,248729,19,"GCH Granules USA, Inc.",1,
3,Oct 17 2022 10:11AM,248719,15,"Alliance Pharma, Inc.",14,
4,Oct 17 2022 10:10AM,248728,19,"NAPP Technologies, LLC",1,
5,Oct 17 2022 10:06AM,248725,20,"Exact-Rx, Inc.",1,
6,Oct 17 2022 10:02AM,248724,12,Hush Hush,1,
7,Oct 17 2022 9:51AM,248712,10,ISDIN Corporation,53,
8,Oct 17 2022 9:40AM,248718,20,"Emersa Waterbox, LLC",1,
9,Oct 17 2022 9:30AM,248715,10,"CLINUVEL, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC",21,
1,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation,59,
2,Oct 17 2022 10:12AM,248729,19,"GCH Granules USA, Inc.",1,
3,Oct 17 2022 10:11AM,248719,15,"Alliance Pharma, Inc.",14,
4,Oct 17 2022 10:10AM,248728,19,"NAPP Technologies, LLC",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC",21.0,NaN
1,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation,59.0,NaN
2,Oct 17 2022 10:12AM,248729,19,"GCH Granules USA, Inc.",1.0,NaN
3,Oct 17 2022 10:11AM,248719,15,"Alliance Pharma, Inc.",14.0,NaN
4,Oct 17 2022 10:10AM,248728,19,"NAPP Technologies, LLC",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 17 2022 10:16AM,248722,15,"Brookfield Pharmaceuticals, LLC",21.0,0.0
1,Oct 17 2022 10:15AM,248723,10,ISDIN Corporation,59.0,0.0
2,Oct 17 2022 10:12AM,248729,19,"GCH Granules USA, Inc.",1.0,0.0
3,Oct 17 2022 10:11AM,248719,15,"Alliance Pharma, Inc.",14.0,0.0
4,Oct 17 2022 10:10AM,248728,19,"NAPP Technologies, LLC",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2487072,147.0,0.0
12,497431,1.0,55.0
15,497441,35.0,0.0
19,497457,2.0,0.0
20,746140,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2487072,147.0,0.0
1,12,497431,1.0,55.0
2,15,497441,35.0,0.0
3,19,497457,2.0,0.0
4,20,746140,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,147.0,0.0
1,12,1.0,55.0
2,15,35.0,0.0
3,19,2.0,0.0
4,20,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,147.0
1,12,Released,1.0
2,15,Released,35.0
3,19,Released,2.0
4,20,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Executing,0.0,55.0,0.0,0.0,0.0
Released,147.0,1.0,35.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Executing,0.0,55.0,0.0,0.0,0.0
1,Released,147.0,1.0,35.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Executing,0.0,55.0,0.0,0.0,0.0
1,Released,147.0,1.0,35.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()